## Оценка стоимости инвестиционной облигации Б-1-116«USDRUB»

### Основная информация по продукту
Структурный продукт имеет две составляющие.<br />
Первая - облигация на 6 месяцев с купонным доходом 0.01% и выплатой при погашении.<br />
Вторая - барьерный пут-опцион на 6 месяцев на валютную пару USD/RUB. Условия опциона таковы: барьер составляет 95% от спот цены(USD/RUB) на момент заключения сделки, соответственно при падении курса доллара к рублю ниже уровня 95% опцион активируется. Пут-опцион покупается за счет дисконта по облигации. <br /> 
Безрзисковую норму доходности возьмем исходя из доходностей ОФЗ на полгода.

### Импортируем необходимые библиотеки

In [175]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from scipy.stats import norm

### Вбиваем базовые данные

In [176]:
nominal = 1000
coupon = 0.0001
risk_free_rate = 0.0340
2.71**0.0071
a = np.exp(1)
T = 1/2

Рассчитаем, какая часть от номинала пойдет на покупку опциона. 

In [186]:
fixed_income_part = nominal/(np.exp(risk_free_rate)) #сколько средств необх положить на fixed income
option_nominal = nominal - fixed_income_part #столько средств останется на покупку опциональности
percentege_of_option_nominal = 100*option_nominal/nominal
print(percentege_of_option_nominal, '% - процентная составляющая номинала, идущая на покупку опциональности')
print(option_nominal, 'Рублей') 
print(fixed_income_part)

3.3428495362493322 % - процентная составляющая номинала, идущая на покупку опциональности
33.42849536249332 Рублей
966.5715046375067


In [159]:
exchange_rates = yf.download('RUB=X','2020-10-02','2021-04-02')
exchange_rates.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-10-01,77.665604,77.743301,76.984100,77.665604,77.665604,0
2020-10-02,77.214401,78.792000,77.198601,77.206200,77.206200,0
2020-10-05,78.139999,78.851997,77.838997,78.139999,78.139999,0
2020-10-06,78.045502,78.641998,77.634003,78.109398,78.109398,0
2020-10-07,78.395103,78.459999,77.930000,78.372803,78.372803,0


## Расчет historical volatility
Скачаиваем данные о курсе USD/RUB

In [178]:
price = yf.download('RUB=X','2019-10-02','2020-10-02')

[*********************100%***********************]  1 of 1 completed


In [179]:
price.head()
price = pd.DataFrame(price)

In [180]:
Close = price['Adj Close']
Close.head()

Date
2019-10-08    64.944099
2019-10-10    65.017303
2019-10-11    64.392700
2019-10-14    64.239601
2019-10-15    64.252403
Name: Adj Close, dtype: float64

In [181]:
log_returns = np.log(Close/Close.shift(1)).dropna()
log_returns

daily_std = log_returns.std()

annualized_std = daily_std * np.sqrt(252)

In [188]:
volatility = annualized_std
print(volatility, '- годовая волатильность за предыдущий период')

0.1768324388355417 - годовая волатильность за предыдущий период


In [183]:
#s - spot price
#k - strike price(=spot в нашем случае)
def BlackScoles(r, S, K, T, sigma, type='C'):
    d1 = ((np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T)))
    d2 = d1 - sigma*np.sqrt(T)
    price = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    if type == "C":
        price = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
    elif type == "P":
        price = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
    return price
   


## Для того чтобы синтезировать барьерный пут-опцион проведем следующие действия:
#### Покупаем пут-опцион со страйковой ценой равной курсу USD/RUB на момент заключения контракта
#### Покупаем колл-опцион со страйковой ценой равной 1/(1.05) от курса USD/RUB на момент заключения контракта
#### Продаем колл опцион со страйковой ценой равной курсу USD/RUB на момент заключения контракта
Все опционы на один и тот же объем. Тогда сложив графики payoff, получим необходимый барьерный пут опцион.


In [184]:
spot = 100
strike = 95
p_call_long =  - BlackScoles(risk_free_rate, spot, strike, T, volatility, type="C")
p_call_short = BlackScoles(risk_free_rate, spot, spot, T, volatility, type="C")
p_put_long = -BlackScoles(risk_free_rate, spot, spot, T, volatility, type="P")
p_sum = p_call_long + p_call_short + p_put_long
print(abs(p_sum), ' - стоимость опциона, %')

7.135998982993861  - стоимость опциона, %


### Как видно, на покупку опциональности можно потратить: 

In [189]:
percentege_of_option_nominal

3.3428495362493322

Вычислим коэффициент участия


In [190]:
participation_coefficient = percentege_of_option_nominal/abs(p_sum)
participation_coefficient

0.4684487125370724